In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import nltk
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from es_lemmatizer import lemmatize
from stop_words import get_stop_words
from nltk.corpus import stopwords
from spacy_spanish_lemmatizer import SpacyCustomLemmatizer
import re
import string
nltk.download('stopwords')
from PIL import features
from wordcloud import WordCloud
nltk.download('punkt')
from textblob import TextBlob
from googletrans import Translator
from sentiment_analysis_spanish import sentiment_analysis
import goslate
import concurrent.futures

[nltk_data] Downloading package wordnet to C:\Users\Rafa
[nltk_data]     OpSeeker\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Rafa
[nltk_data]     OpSeeker\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Rafa
[nltk_data]     OpSeeker\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Exploramos datos

In [3]:
data=pd.read_csv('output/all_data_news.csv',error_bad_lines=False)
data.head()

,Unnamed: 0,title,text,authors,top_image,movies,journal,published,article_link,keywords,summary,html
0,0,"Últimas noticias del coronavirus, en directo |...",Nota a los lectores: EL PAÍS ofrece en abierto...,"['El País', 'Cati Cladera', 'Efe', 'Vídeo']",https://imagenes.elpais.com/resizer/D3KEdA43-x...,[],elpais,2020-12-15 00:00:00,https://elpais.com/sociedad/2020-12-15/ultimas...,[],NaN,"<!DOCTYPE html><html lang=""es""><head><title>Úl..."
1,1,Bruselas podrá exigir la venta de parte de su ...,Bruselas ha colocado este martes las bases de ...,"['Lluís Pellicer', 'Pool']",https://imagenes.elpais.com/resizer/IXP_qZSOq9...,[],elpais,2020-12-15 00:00:00,https://elpais.com/economia/2020-12-15/brusela...,[],NaN,"<!DOCTYPE html><html lang=""es""><head><title>Br..."
2,2,El Gobierno del Reino Unido impondrá multas mi...,El Gobierno del Reino Unido ha anunciado final...,"['Rafa De Miguel', 'Andy Rain']",https://imagenes.elpais.com/resizer/U8iZddH13M...,[],elpais,2020-12-15 00:00:00,https://elpais.com/tecnologia/2020-12-15/el-go...,[],NaN,"<!DOCTYPE html><html lang=""es""><head><title>El..."
3,3,¿Y si Canarias fuera la fuente de tierras rara...,"“Canarias no es solo turismo de sol y playa, n...","['Guillermo Vega', 'Kate Passileggeri', 'Eyeem']",https://imagenes.elpais.com/resizer/qYySQ1Qzfd...,[],elpais,2020-12-14 00:00:00,https://elpais.com/tecnologia/2020-12-14/y-si-...,[],NaN,"<!DOCTYPE html><html lang=""es""><head><title>¿Y..."
4,4,Pornhub borra dos tercios de los vídeos de su ...,Pornhub cede a las presiones y borra los vídeo...,"['Jaime Porras Ferreyra', 'Getty Images', 'M. ...",https://rt00.epimg.net/retina/imagenes/2020/12...,[],elpais,2020-12-15 00:00:00,https://retina.elpais.com/retina/2020/12/14/in...,[],NaN,"<!DOCTYPE html>\n<html lang=""es"">\n<head>\n<me..."


In [4]:
data.size

85980

In [5]:
# Borramos columnas que no necesitamos y datos duplicados

In [6]:
data=data.drop(['Unnamed: 0', 'top_image', 'movies', 'published', 'article_link', 'keywords' , 'summary', 'html'],axis=1)
data = data.drop_duplicates()
data.size

23032

In [7]:
data.head()

,title,text,authors,journal
0,"Últimas noticias del coronavirus, en directo |...",Nota a los lectores: EL PAÍS ofrece en abierto...,"['El País', 'Cati Cladera', 'Efe', 'Vídeo']",elpais
1,Bruselas podrá exigir la venta de parte de su ...,Bruselas ha colocado este martes las bases de ...,"['Lluís Pellicer', 'Pool']",elpais
2,El Gobierno del Reino Unido impondrá multas mi...,El Gobierno del Reino Unido ha anunciado final...,"['Rafa De Miguel', 'Andy Rain']",elpais
3,¿Y si Canarias fuera la fuente de tierras rara...,"“Canarias no es solo turismo de sol y playa, n...","['Guillermo Vega', 'Kate Passileggeri', 'Eyeem']",elpais
4,Pornhub borra dos tercios de los vídeos de su ...,Pornhub cede a las presiones y borra los vídeo...,"['Jaime Porras Ferreyra', 'Getty Images', 'M. ...",elpais


In [8]:
data.groupby(['journal']).count()

,title,text,authors
journal,,,
20minutos,193,193,193
abc,223,219,223
elcorreo,163,131,163
eldiario,788,786,788
elmundo,314,286,314
elpais,731,719,731
elperiodico,254,254,254
lavanguardia,748,744,748
lavozdegalicia,1732,1638,1732


In [21]:
# limpiamos datos. 


In [10]:
def clean_text_one(text):
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [11]:
data.title = data.title.apply(clean_text_one)

In [14]:
data['text'] = data['text'].astype(str)

data.text = data.text.apply(clean_text_one)

In [15]:
data.head()

,title,text,authors,journal
0,Últimas noticias del coronavirus en directo ...,Nota a los lectores EL PAÍS ofrece en abierto...,"['El País', 'Cati Cladera', 'Efe', 'Vídeo']",elpais
1,Bruselas podrá exigir la venta de parte de su ...,Bruselas ha colocado este martes las bases de ...,"['Lluís Pellicer', 'Pool']",elpais
2,El Gobierno del Reino Unido impondrá multas mi...,El Gobierno del Reino Unido ha anunciado final...,"['Rafa De Miguel', 'Andy Rain']",elpais
3,¿Y si Canarias fuera la fuente de tierras rara...,“Canarias no es solo turismo de sol y playa n...,"['Guillermo Vega', 'Kate Passileggeri', 'Eyeem']",elpais
4,Pornhub borra dos tercios de los vídeos de su ...,Pornhub cede a las presiones y borra los vídeo...,"['Jaime Porras Ferreyra', 'Getty Images', 'M. ...",elpais


In [16]:
def clean_text_two(text):
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [17]:
data.title = data.title.apply(clean_text_two)

In [18]:
data.text = data.text.apply(clean_text_two)

In [19]:
data.head()

,title,text,authors,journal
0,Últimas noticias del coronavirus en directo ...,Nota a los lectores EL PAÍS ofrece en abierto...,"['El País', 'Cati Cladera', 'Efe', 'Vídeo']",elpais
1,Bruselas podrá exigir la venta de parte de su ...,Bruselas ha colocado este martes las bases de ...,"['Lluís Pellicer', 'Pool']",elpais
2,El Gobierno del Reino Unido impondrá multas mi...,El Gobierno del Reino Unido ha anunciado final...,"['Rafa De Miguel', 'Andy Rain']",elpais
3,¿Y si Canarias fuera la fuente de tierras rara...,“Canarias no es solo turismo de sol y playa n...,"['Guillermo Vega', 'Kate Passileggeri', 'Eyeem']",elpais
4,Pornhub borra dos tercios de los vídeos de su ...,Pornhub cede a las presiones y borra los vídeo...,"['Jaime Porras Ferreyra', 'Getty Images', 'M. ...",elpais


In [20]:
data.to_csv("output/clean_data.csv")


In [ ]:
list_keywords_coronavirus = ["Coronavirus", "coronavirus", "covid", "Covid", "pandemia", "Pandemia", "vacuna", "Vacuna",
                             "virus", "Virus", "Segunda ola", "segunda ola", "vacunación", "Vacunación", "Vacunar", "vacunar",
                            "contagios", "contagiados", "Contagiados", "Contagios"]

In [ ]:
# sLength = len(data['text'])
# data['coronavirus'] = pd.Series(np.random.randn(sLength), index=data.index)
data['title'] = data['title'].astype(str)

def isCoronavirus(text):
        if any(word in text for word in list_keywords_coronavirus):
            return 1
        else:
            return 0

In [ ]:
data['isArticleCoronavirus'] = data["title"].apply(isCoronavirus)


In [ ]:
data_covid = data[data.isArticleCoronavirus == 1]

In [ ]:
data_covid.size

In [ ]:
z=data_covid.groupby(by=['journal'],as_index=False).count().sort_values(by='isArticleCoronavirus',ascending=False).head(10)

plt.figure(figsize=(10,5))
sns.set_style("whitegrid")
ax = sns.barplot(x=z['journal'],y=z['isArticleCoronavirus'], data=z)
plt.xlabel("isArticleCoronavirus")
plt.ylabel("Count")
plt.title("Counts isArticleCoronavirus per journal")

In [ ]:
list_keywords_monarchy = ["Juan Carlos I", "Juan Carlos", "emérito", "Don Juan Carlos", "El Rey"]


In [ ]:
def isMonarchy(text):
        if any(word in text for word in list_keywords_monarchy):
            return 1
        else:
            return 0

In [ ]:
data['isArticleMonarchy'] = data["title"].apply(isMonarchy)

In [ ]:
data_monarchy = data[data.isArticleMonarchy == 1]

In [ ]:
data_monarchy.head()

In [ ]:
z=data_monarchy.groupby(by=['journal'],as_index=False).count().sort_values(by='isArticleMonarchy',ascending=False).head(10)

plt.figure(figsize=(10,5))
sns.set_style("whitegrid")
ax = sns.barplot(x=z['journal'],y=z['isArticleMonarchy'], data=z)
plt.xlabel("isArticleMonarchy")
plt.ylabel("Count")
plt.title("Counts isArticleMonarchy per journal")

In [ ]:
list_keywords_psoe_sanchez= ["Sánchez", "Pedro Sánchez", "Psoe", "Partido Socialista"]
list_keywords_vox_abascal= ['Santiago Abascal', "Abascal", "Vox"]
list_keywords_podemos_iglesias= ['Pablo Iglesias', 'Iglesias', "Podemos", "Unidas Podemos"]
list_keywords_pp_casado= ['Pablo Casado', 'Casado', "PP", "Partido Popular"]
list_keywords_ciudadanos_ines= ['Inés Arrimadas', 'Arrimadas', "Ciudadanos", "Cs"]


def probPolitical(text):
    isSanchez = 0 
    isCasado = 0 
    isArrimadas = 0
    isAbascal = 0
    isIglesias = 0

    if any(word in text for word in list_keywords_psoe_sanchez):
        isSanchez += 1 
    if any(word in text for word in list_keywords_vox_abascal):
         isAbascal += 1 
    if any(word in text for word in list_keywords_podemos_iglesias):
         isIglesias += 1 
    if any(word in text for word in list_keywords_pp_casado):
        isCasado += 1 
    if any(word in text for word in list_keywords_ciudadanos_ines):
        isArrimadas += 1 

    total = isIglesias + isAbascal + isIglesias + isCasado + isArrimadas

    if (total > 0):
        return isCasado/total #cambiar politico
    else:
        return 0
    
        

In [ ]:
data['isIglesias'] = data["title"].apply(probPolitical)
data['isIglesias'] 

In [ ]:

data_isIglesias2 = data[data.isIglesias > 0.5]
data_isIglesias2.head()

In [ ]:
z=data_isIglesias.groupby(by=['journal'],as_index=False).count().sort_values(by='isIglesias',ascending=False).head(10)

plt.figure(figsize=(10,5))
sns.set_style("whitegrid")
ax = sns.barplot(x=z['journal'],y=z['isSanchez'], data=z)
plt.xlabel("isArticleMonarchy")
plt.ylabel("Count")
plt.title("Counts isArticleMonarchy per journal")

In [ ]:
data_isSanchez.head()

In [ ]:
data_monarchy['title'] = data_monarchy['title'].astype(str)
data_monarchy['text'] = data_monarchy['text'].astype(str)



In [ ]:
import time
from deep_translator import GoogleTranslator


def translate_comment(x):
    time.sleep(3)
    text_en = GoogleTranslator(source='es', target='en').translate(x)
    return text_en


In [ ]:
data['title_en'] = data["title"].apply(translate_comment)



In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def sentimentAnalysis(sentence):
    sia = SentimentIntensityAnalyzer()
    polarity = sia.polarity_scores(sentence)
    pol = polarity['compound']
    return pol

In [ ]:
data_isSanchez['sentiment_compound'] = data_isSanchez['text'].apply(sentimentAnalysis)
data_isSanchez.head()



In [ ]:
data_isSanchez.groupby(['journal'])['sentiment_compound'].mean()

In [ ]:
data_monarchy['sentiment_compound'] = data_monarchy['text'].apply(sentimentAnalysis)
data_monarchy.head()

In [ ]:
data_monarchy.groupby(['journal'])['sentiment_compound'].mean()

In [ ]:
data_isIglesias['title'] = data_isIglesias["title"].apply(translate_comment)


In [ ]:
data_isIglesias['title'] = data_isIglesias['title'].astype(str)
data_isIglesias['title'] = data_isIglesias['title'].astype(str)


data_isIglesias['sentiment_compound'] = data_isIglesias['title'].apply(sentimentAnalysis)


In [ ]:
data_isIglesias.groupby(['journal'])['sentiment_compound'].mean()

In [ ]:
from sentiment_analysis_spanish import sentiment_analysis
def sentimentSpanish(text):  
    sentiment = sentiment_analysis.SentimentAnalysisSpanish()
    return sentiment.sentiment(text)

In [ ]:
data_isIglesias['text'] = data_isIglesias['text'].astype(str)

data_isIglesias['sentiment_compound_es_text'] = data_isIglesias['text'].apply(sentimentAnalysis)


In [ ]:
data_isIglesias.groupby(['journal'])['sentiment_compound_es_text'].mean()
data_isIglesias.head()

In [ ]:
from textblob import TextBlob
    
pol = lambda x: TextBlob(x).sentiment.polarity    

pol2 = lambda x: x.sentiment.polarity

def traduce (x):
    time.sleep(1)
    return TextBlob(x).translate(to="en")

data['title_es'] = data['title'].apply(traduce)


In [ ]:
data_isIglesias2['polarity'] = data_isIglesias2['traduce'].apply(pol2)


In [ ]:
data_isIglesias2

In [ ]:
data_isIglesias2.groupby(['journal'])['polarity'].mean()
